---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Take away
- read txt file
- regular expression to extract date patter in pure txt file
    - Mar-20-2009 -> (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[- ]\d{1,2}[- ]\d{4}
- get Month abbreviation and full name using import calendar
    - list(set(list(calendar.month_name)+list(calendar.month_abbr))) -> whole list of month
- replace Month name using dictionary method -> ex: Jan -> 1; January -> 1
    - cal_dict = dict((name, num) for num, name in enumerate(list(calendar.month_name)))
    - cal_dict2 = dict((name, num) for num, name in enumerate(list(calendar.month_abbr)))
    - data["month"] = data["month"].replace(cal_dict)
    - data["month"] = data["month"].replace(cal_dict2)
- rank the date (recent get larger, older date get less number)
    - data["rank"] = data.rank(ascending=True, method='dense').astype(int)





# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re
import numpy as np
import calendar

/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def date_sorter():
    
    
    #print("---> 1. read txt file")
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    data = pd.DataFrame(doc, columns = ["text"])
    

    #print("---> 2. find date info in raw txt file")

    def find_date(row):
        # for 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        # row[72] has two items
        if len(re.findall("\d{1,2}[/]\d{1,2}[/](?:\d{2,4})", row["text"]))!=0:
            if len(re.findall("\d{1,2}[/]\d{1,2}[/](?:\d{2,4})", row["text"]))==1:
                return re.findall("\d{1,2}[/]\d{1,2}[/](?:\d{2,4})", row["text"])
            else:
                return "check"

        # for Mar-20-2009 & Mar 20 2009
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[- ]\d{1,2}[- ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[- ]\d{1,2}[- ]\d{4}", row["text"])

        # for August 12 2004
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{1,2}[ ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{1,2}[ ]\d{4}", row["text"])

        # for Mar 20, 2009; March 20, 2009; Mar. 20, 2009
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[- ]\d{1,2}[,][- ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.]?[- ]\d{1,2}[,][- ]\d{4}", row["text"])

        # for 20 Mar 2009; 20 Mar. 2009; 20 March, 2009
        elif len(re.findall("\d{1,2}[ ](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?[ ]\d{4}", row["text"]))!=0:
            return re.findall("\d{1,2}[ ](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]?[ ]\d{4}", row["text"])

        # for Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{1,2}(?:th|st|nd)[,][ ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{1,2}(?:th|st|nd)[,][ ]\d{4}", row["text"])

        # for Feb 2009; Sep 2009; Oct 2010
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]\d{4}", row["text"])

        # for 4-13-82
        elif len(re.findall("\d{1,2}[-]\d{1,2}[-]\d{1,2}", row["text"]))!=0:
            return re.findall("\d{1,2}[-]\d{1,2}[-]\d{1,2}", row["text"])


        # for  May, 2011
        elif len(re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,][ ]\d{4}", row["text"]))!=0:
            return re.findall("(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,][ ]\d{4}", row["text"])

        # for 6/2008; 12/2009
        elif len(re.findall("\d{1,2}[/]\d{4}", row["text"]))!=0:
            return re.findall("\d{1,2}[/]\d{4}", row["text"])

        # 2009; 2010
        elif len(re.findall("\d{4}", row["text"]))!=0:
            return re.findall("\d{4}", row["text"])


        else:
            return 0

    try:
        data["extract_date"] = data.apply(find_date, axis=1) 
    except Exception as e:
        print(e)    
    
 
    #print("---> 2.1 deal with special case -> have two items in a list")
    row_num = int(data[data["extract_date"]=="check"].index[0])
    row = data.loc[row_num]
    data.loc[row_num,"extract_date"] = re.findall("\d{1,2}[/]\d{1,2}[/](?:\d{2,4})", row["text"])[0]


    
    #print("---> 3. separate raw extract date as month, day, year")
    data["result_1"] = data.apply(lambda row: re.split("/", row["extract_date"]), axis= 1)
    data["result_2"] = data.apply(lambda row: re.split("-", row["extract_date"]), axis= 1)
    data["result_3"] = data.apply(lambda row: re.split(" ", row["extract_date"]), axis= 1)    
    
    
    
    #print("---> 3.1 more item in the result mean it has dd,mm,yy instead of one item as dd-mm-yy")
    month_list = list(set(list(calendar.month_name)+list(calendar.month_abbr)))
    def find_large_num(row):
        if len(row["result_1"])>len(row["result_2"]) and len(row["result_1"])>len(row["result_3"]):
            return row["result_1"]
        elif len(row["result_2"])>len(row["result_1"]) and len(row["result_2"])>len(row["result_3"]):
            return row["result_2"]
        elif len(row["result_3"])>len(row["result_1"]) and len(row["result_3"])>len(row["result_2"]):
            return row["result_3"]

        # for result_1 = result_2 = result_3 -> return anyone of them
        else:
            return row["result_1"]

    data["final"] = data.apply(find_large_num, axis=1)
    del data["result_1"], data["result_2"], data["result_3"]
    data["len"] = data["final"].apply(len)    
    
    

    
    #print("---> 3.2 get month, day and year")

    # deal with special cases -> [24, Jan, 2001] -> DD/MM/YYYY
    def get_month(row):
        if row["len"]>2 and [re.sub("\.|\,", "", i) for i in row["final"]][1] in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][1]
        elif row["len"]>2 and [re.sub("\.|\,", "", i) for i in row["final"]][1] not in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][0]
        elif row["len"]==2 and [re.sub("\.|\,", "", i) for i in row["final"]][0] in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][0]
        elif row["len"]==2 and [re.sub("\.|\,", "", i) for i in row["final"]][1] in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][1]
        else:
            return "01"

    def get_day(row):
        if row["len"]>2 and [re.sub("\.|\,", "", i) for i in row["final"]][1] in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][0]
        elif row["len"]>2 and [re.sub("\.|\,", "", i) for i in row["final"]][1] not in  month_list:
            return [re.sub("\.|\,", "", i) for i in row["final"]][1]
        else:
            return "01"   

    data["month"] = data.apply(get_month, axis=1)
    data["day"] = data.apply(get_day, axis=1)

    # note consider case like 12 Dec 2019
    #data["month"] = data.apply(lambda row: [re.sub("\.|\,", "", i) for i in row["final"]][0] if row["len"]>=2 else "01" , axis = 1)              
    #data["day"] = data.apply(lambda row: [re.sub("\.|\,", "", i) for i in row["final"]][1] if row["len"]>2 else "01" , axis = 1)


    def find_year(row):
        if row["len"]==3:
            return [re.sub("\.|\,", "", i) for i in row["final"]][2]
        elif row["len"]==2:
            return [re.sub("\.|\,", "", i) for i in row["final"]][1]
        else:
            return [re.sub("\.|\,", "", i) for i in row["final"]][0]

    data["year"] = data.apply(find_year, axis=1)          
    data["len_year"] = data["year"].apply(len)
    data["revised_year"] = data.apply(lambda row: "19"+row["year"] if row["len_year"]==2 else row["year"], axis=1)
    del data["len"], data["year"], data["len_year"]


    # replace with dictionary
    cal_dict = dict((name, num) for num, name in enumerate(list(calendar.month_name)))
    cal_dict2 = dict((name, num) for num, name in enumerate(list(calendar.month_abbr)))
    data["month"] = data["month"].replace(cal_dict)
    data["month"] = data["month"].replace(cal_dict2)
    data["month"].unique()


    #print("---> 3.3 fix typo problems")
    # get typo of month
    data["month"] = data["month"].replace('Janaury', '01')
    data["month"] = data["month"].replace('Decemeber', '12')    
    
    
    #print("---> 3.4 make day and month as two digit")
    data["month"] = data.apply(lambda row: "%02d" %int(row["month"]), axis=1)
    data["day"] = data.apply(lambda row: "%02d" %int(row["day"]), axis=1)    
   

    #print("---> 4. combine year-month-day")
    data["date"] = data.apply(lambda row: row["revised_year"]+"-"+row["month"]+"-"+row["day"], axis=1)


    #print("---> 5. rank the data)
    # older date has smallest number
    data["rank"] = data["date"].rank(ascending=True, method='dense').astype(int)
    data = data.sort_values(by="rank", ascending = "True")
    data = data.reset_index()
    data = data[["index"]]
    data = data.rename(columns={"index":"index_of_original_dataset"})
    
    return pd.Series(data["index_of_original_dataset"], index=data.index)

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     415
16     380
17     422
18     405
19     375
20     345
21     486
22     335
23      36
24     323
25      57
26     351
27     481
28     402
29     436
      ... 
470     34
471    220
472    208
473    243
474    139
475    320
476    431
477    480
478    244
479    286
480    279
481    198
482    439
483    463
484    366
485    401
486    381
487    255
488    475
489    257
490    152
491    235
492    413
493    464
494    427
495    253
496    231
497    141
498    186
499    161
Name: index_of_original_dataset, Length: 500, dtype: int64

In [3]:
#####################################
# reference code
#####################################

def date_sorter():
    
    #print("---> 1. read txt file")
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)           
    dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))')
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    del dates_extracted[3]
    del dates_extracted[4]
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))'))
    del dates_extracted[3]
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Without day
    dates_without_day = df[index_left].str.extractall('(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))')
    dates_without_day = dates_without_day.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))'))
    dates_without_day['day'] = 1
    dates_extracted = dates_extracted.append(dates_without_day)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Only year
    dates_only_year = df[index_left].str.extractall(r'(?P<origin>(?P<year>\d{4}))')
    dates_only_year['day'] = 1
    dates_only_year['month'] = 1
    dates_extracted = dates_extracted.append(dates_only_year)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Year
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: str(x))

    # Month
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    dates_extracted.replace({"month": month_dict}, inplace=True)
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: str(x))

    # Day
    dates_extracted['day'] = dates_extracted['day'].apply(lambda x: str(x))

    # Cleaned date
    dates_extracted['date'] = dates_extracted['month'] + '/' + dates_extracted['day'] + '/' + dates_extracted['year']
    dates_extracted['date'] = pd.to_datetime(dates_extracted['date'])

    dates_extracted.sort_values(by='date', inplace=True)
    df1 = pd.Series(list(dates_extracted.index.labels[0]))    
    
    return df1
#date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    286
477    244
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64